In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from textblob import TextBlob
import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN
from tensorflow.keras import Model
import nltk
from nltk.corpus import stopwords
import time
import os

# 下载NLTK停用词
nltk.download('stopwords')

class RecommendationSystem:
    def __init__(self):
        # 配置参数
        self.config = {
            'time_decay_lambda': 0.2,
            'min_user_interactions': 5,
            'min_item_interactions': 10,
            'top_k': 10,
            'svd_factors': 50,
            'gcn_embedding_size': 128,
            'test_size': 0.2,
            'random_state': 42,
            'tfidf_max_features': 500
        }
        
        # 数据存储
        self.df = None
        self.train_data = None
        self.test_data = None
        
        # 模型存储
        self.user_cf_model = None
        self.item_cf_model = None
        self.svdpp_model = None
        self.hybrid_model = None
        
        # 结果存储
        self.results = {}
        
        # 创建结果目录
        os.makedirs('results', exist_ok=True)
        os.makedirs('visualizations', exist_ok=True)

    def load_data(self, movielens_path, amazon_path):
        """加载并合并MovieLens和Amazon数据集"""
        print("Loading and merging datasets...")
        
        # 加载MovieLens数据
        movie_data = pd.read_csv(movielens_path)
        
        # 加载Amazon数据
        amazon_data = pd.read_csv(amazon_path)
        
        # 创建统一评分体系
        combined_data = pd.concat([
            movie_data[['userId', 'movieId', 'rating', 'timestamp']].rename(
                columns={'movieId': 'itemId'}
            ),
            amazon_data[['reviewerID', 'asin', 'overall', 'reviewText', 'unixReviewTime']].rename(
                columns={'reviewerID': 'userId', 'asin': 'itemId', 'overall': 'rating', 
                         'unixReviewTime': 'timestamp'}
            )
        ], ignore_index=True)
        
        # 转换时间戳
        combined_data['timestamp'] = pd.to_datetime(combined_data['timestamp'], unit='s')
        
        # 填充缺失的reviewText
        combined_data['reviewText'] = combined_data['reviewText'].fillna('')
        
        self.df = combined_data
        print(f"Loaded {len(self.df)} records with {self.df['userId'].nunique()} users and {self.df['itemId'].nunique()} items")
        
        return self.df

    def detect_anomalies(self):
        """检测并处理异常数据"""
        print("Detecting anomalies...")
        
        # 检测同一用户1分钟内产生超过10条记录
        df_sorted = self.df.sort_values(['userId', 'timestamp'])
        df_sorted['time_diff'] = df_sorted.groupby('userId')['timestamp'].diff().dt.total_seconds().fillna(0)
        
        # 标记异常用户
        anomaly_users = set()
        for user, group in df_sorted.groupby('userId'):
            # 计算滚动窗口内的交互次数
            rolling_count = group['time_diff'].rolling(window=10, min_periods=1).apply(
                lambda x: np.sum(x < 60), raw=False
            )
            if any(rolling_count > 10):
                anomaly_users.add(user)
        
        # 删除异常用户的所有记录
        original_count = len(self.df)
        self.df = self.df[~self.df['userId'].isin(anomaly_users)]
        removed_count = original_count - len(self.df)
        
        print(f"Removed {removed_count} records from {len(anomaly_users)} anomaly users")
        
        return self.df

    def normalize_ratings(self):
        """标准化评分到[0,1]区间"""
        print("Normalizing ratings...")
        
        # MovieLens评分标准化 (1-5 -> 0-1)
        movielens_mask = self.df['itemId'].str.startswith('m')  # 假设MovieLens ID以'm'开头
        self.df.loc[movielens_mask, 'rating_norm'] = (self.df.loc[movielens_mask, 'rating'] - 1) / 4
        
        # Amazon评分标准化 (1-5 -> 0.2-1.0)
        amazon_mask = self.df['itemId'].str.startswith('B')  # 假设Amazon ASIN以'B'开头
        self.df.loc[amazon_mask, 'rating_norm'] = self.df.loc[amazon_mask, 'rating'] / 5
        
        return self.df

    def apply_time_decay(self):
        """应用时间衰减因子"""
        print("Applying time decay...")
        
        current_time = self.df['timestamp'].max()
        time_diff = (current_time - self.df['timestamp']).dt.total_seconds() / 86400  # 转换为天
        self.df['time_weight'] = np.exp(-self.config['time_decay_lambda'] * time_diff)
        self.df['weighted_rating'] = self.df['rating_norm'] * self.df['time_weight']
        
        # 可视化时间衰减权重分布
        plt.figure(figsize=(10, 6))
        sns.kdeplot(self.df['time_weight'], fill=True)
        plt.title('时间衰减权重分布', fontsize=14)
        plt.xlabel('衰减权重', fontsize=12)
        plt.ylabel('密度', fontsize=12)
        plt.savefig('visualizations/time_weight_dist.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        return self.df

    def extract_text_features(self):
        """提取文本特征和情感分析"""
        print("Extracting text features...")
        
        # 情感分析
        self.df['sentiment'] = self.df['reviewText'].apply(
            lambda x: TextBlob(str(x)).sentiment.polarity
        )
        
        # TF-IDF特征提取
        tfidf = TfidfVectorizer(
            stop_words=stopwords.words('english'),
            max_features=self.config['tfidf_max_features']
        )
        
        # 只对Amazon评论应用TF-IDF
        amazon_mask = self.df['itemId'].str.startswith('B')
        text_features = tfidf.fit_transform(self.df.loc[amazon_mask, 'reviewText'])
        
        # 创建TF-IDF DataFrame
        tfidf_df = pd.DataFrame(
            text_features.toarray(), 
            columns=[f"tfidf_{col}" for col in tfidf.get_feature_names_out()],
            index=self.df[amazon_mask].index
        )
        
        # 合并回主数据框
        self.df = pd.concat([self.df, tfidf_df], axis=1)
        
        # 填充NaN值为0
        tfidf_cols = [col for col in self.df.columns if col.startswith('tfidf_')]
        self.df[tfidf_cols] = self.df[tfidf_cols].fillna(0)
        
        return self.df

    def filter_sparse_data(self):
        """过滤交互次数过少的用户和商品"""
        print("Filtering sparse data...")
        
        # 计算用户交互次数
        user_counts = self.df['userId'].value_counts()
        # 计算商品交互次数
        item_counts = self.df['itemId'].value_counts()
        
        # 过滤交互次数过少的用户和商品
        filtered_users = user_counts[user_counts >= self.config['min_user_interactions']].index
        filtered_items = item_counts[item_counts >= self.config['min_item_interactions']].index
        
        original_size = len(self.df)
        self.df = self.df[
            self.df['userId'].isin(filtered_users) & 
            self.df['itemId'].isin(filtered_items)
        ]
        
        print(f"Filtered {original_size - len(self.df)} sparse records")
        print(f"Remaining data: {len(self.df)} records, {self.df['userId'].nunique()} users, {self.df['itemId'].nunique()} items")
        
        return self.df

    def generate_graph_embeddings(self):
        """使用GCN生成图嵌入"""
        print("Generating graph embeddings with GCN...")
        
        # 创建StellarGraph对象
        graph = sg.StellarDiGraph()
        
        # 添加节点
        users = self.df['userId'].unique()
        items = self.df['itemId'].unique()
        
        graph.add_nodes_from(users, node_type="user")
        graph.add_nodes_from(items, node_type="item")
        
        # 添加边
        edges = [(row['userId'], row['itemId']) for _, row in self.df.iterrows()]
        graph.add_edges_from(edges, edge_type="interaction")
        
        # 创建GCN模型
        generator = FullBatchNodeGenerator(graph, method="gcn")
        gcn = GCN(
            layer_sizes=[self.config['gcn_embedding_size']], 
            generator=generator,
            activations=["relu"]
        )
        
        # 构建模型
        x_in, x_out = gcn.in_out_tensors()
        model = Model(inputs=x_in, outputs=x_out)
        
        # 训练模型（这里简化了训练过程）
        # 在实际应用中，需要更复杂的训练过程
        embeddings = model.predict(generator.flow(graph.nodes()))
        
        # 提取用户嵌入
        user_embeddings = embeddings[0][:len(users)]
        user_embedding_df = pd.DataFrame(
            user_embeddings,
            index=users,
            columns=[f"gcn_embed_{i}" for i in range(self.config['gcn_embedding_size'])]
        )
        
        # 提取商品嵌入
        item_embeddings = embeddings[0][len(users):]
        item_embedding_df = pd.DataFrame(
            item_embeddings,
            index=items,
            columns=[f"gcn_embed_{i}" for i in range(self.config['gcn_embedding_size'])]
        )
        
        # 合并嵌入到主数据框
        self.df = self.df.merge(user_embedding_df, left_on='userId', right_index=True, how='left')
        self.df = self.df.merge(item_embedding_df, left_on='itemId', right_index=True, how='left')
        
        # 填充NaN值为0
        gcn_cols = [col for col in self.df.columns if col.startswith('gcn_embed_')]
        self.df[gcn_cols] = self.df[gcn_cols].fillna(0)
        
        print(f"Generated GCN embeddings with {self.config['gcn_embedding_size']} dimensions")
        
        return self.df

    def split_data(self):
        """划分训练集和测试集"""
        print("Splitting data into train and test sets...")
        
        # 按时间划分：最后20%时间的数据作为测试集
        self.df = self.df.sort_values('timestamp')
        split_idx = int(len(self.df) * (1 - self.config['test_size']))
        
        self.train_data = self.df.iloc[:split_idx]
        self.test_data = self.df.iloc[split_idx:]
        
        print(f"Train set: {len(self.train_data)} records")
        print(f"Test set: {len(self.test_data)} records")
        
        return self.train_data, self.test_data

    def create_interaction_matrix(self, data):
        """创建用户-商品交互矩阵"""
        # 创建用户和商品的映射
        unique_users = data['userId'].unique()
        unique_items = data['itemId'].unique()
        
        user_to_idx = {user: idx for idx, user in enumerate(unique_users)}
        item_to_idx = {item: idx for idx, item in enumerate(unique_items)}
        
        # 创建交互矩阵
        rows = data['userId'].map(user_to_idx)
        cols = data['itemId'].map(item_to_idx)
        values = data['weighted_rating']
        
        interaction_matrix = coo_matrix(
            (values, (rows, cols)),
            shape=(len(unique_users), len(unique_items))
        ).tocsr()
        
        return interaction_matrix, user_to_idx, item_to_idx, unique_users, unique_items

    class UserCF:
        """基于用户的协同过滤"""
        def __init__(self, k=10, similarity_threshold=0.3):
            self.k = k
            self.similarity_threshold = similarity_threshold
            self.interaction_matrix = None
            self.user_to_idx = None
            self.idx_to_user = None
            self.item_to_idx = None
            self.idx_to_item = None
        
        def fit(self, interaction_matrix, user_to_idx, item_to_idx):
            self.interaction_matrix = interaction_matrix
            self.user_to_idx = user_to_idx
            self.idx_to_user = {v: k for k, v in user_to_idx.items()}
            self.item_to_idx = item_to_idx
            self.idx_to_item = {v: k for k, v in item_to_idx.items()}
            
            # 计算用户相似度矩阵
            self.user_similarity = cosine_similarity(interaction_matrix)
            
            # 应用阈值
            self.user_similarity[self.user_similarity < self.similarity_threshold] = 0
        
        def predict(self, user_id, n=10):
            if user_id not in self.user_to_idx:
                return []
            
            user_idx = self.user_to_idx[user_id]
            
            # 获取相似用户
            similar_users = np.argsort(self.user_similarity[user_idx])[::-1][1:self.k+1]
            
            # 计算推荐分数
            scores = np.zeros(self.interaction_matrix.shape[1])
            
            for sim_user_idx in similar_users:
                similarity = self.user_similarity[user_idx, sim_user_idx]
                if similarity > 0:
                    scores += similarity * self.interaction_matrix[sim_user_idx].toarray().flatten()
            
            # 排除用户已经交互过的商品
            interacted_items = self.interaction_matrix[user_idx].indices
            scores[interacted_items] = -np.inf
            
            # 获取top N推荐
            top_item_indices = np.argsort(scores)[::-1][:n]
            return [self.idx_to_item[idx] for idx in top_item_indices if scores[idx] > 0]

    class ItemCF:
        """基于物品的协同过滤"""
        def __init__(self, k=10, similarity_threshold=0.3):
            self.k = k
            self.similarity_threshold = similarity_threshold
            self.interaction_matrix = None
            self.user_to_idx = None
            self.idx_to_user = None
            self.item_to_idx = None
            self.idx_to_item = None
        
        def fit(self, interaction_matrix, user_to_idx, item_to_idx):
            self.interaction_matrix = interaction_matrix
            self.user_to_idx = user_to_idx
            self.idx_to_user = {v: k for k, v in user_to_idx.items()}
            self.item_to_idx = item_to_idx
            self.idx_to_item = {v: k for k, v in item_to_idx.items()}
            
            # 计算物品相似度矩阵
            self.item_similarity = cosine_similarity(interaction_matrix.T)
            
            # 应用阈值
            self.item_similarity[self.item_similarity < self.similarity_threshold] = 0
        
        def predict(self, user_id, n=10):
            if user_id not in self.user_to_idx:
                return []
            
            user_idx = self.user_to_idx[user_id]
            
            # 获取用户交互过的商品
            interacted_items = self.interaction_matrix[user_idx].indices
            
            # 计算推荐分数
            scores = np.zeros(self.interaction_matrix.shape[1])
            
            for item_idx in interacted_items:
                # 获取相似商品
                similar_items = np.argsort(self.item_similarity[item_idx])[::-1][1:self.k+1]
                
                for sim_item_idx in similar_items:
                    similarity = self.item_similarity[item_idx, sim_item_idx]
                    if similarity > 0:
                        scores[sim_item_idx] += similarity * self.interaction_matrix[user_idx, item_idx]
            
            # 排除用户已经交互过的商品
            scores[interacted_items] = -np.inf
            
            # 获取top N推荐
            top_item_indices = np.argsort(scores)[::-1][:n]
            return [self.idx_to_item[idx] for idx in top_item_indices if scores[idx] > 0]

    class SVDpp:
        """SVD++矩阵分解模型"""
        def __init__(self, n_factors=50, n_epochs=20, lr=0.005, reg=0.02):
            self.n_factors = n_factors
            self.n_epochs = n_epochs
            self.lr = lr
            self.reg = reg
            self.user_factors = None
            self.item_factors = None
            self.user_biases = None
            self.item_biases = None
            self.global_bias = None
        
        def fit(self, interaction_matrix):
            # 转换为COO格式
            coo_matrix = interaction_matrix.tocoo()
            rows = coo_matrix.row
            cols = coo_matrix.col
            values = coo_matrix.data
            
            n_users, n_items = interaction_matrix.shape
            
            # 初始化参数
            self.global_bias = np.mean(values)
            self.user_biases = np.zeros(n_users)
            self.item_biases = np.zeros(n_items)
            self.user_factors = np.random.normal(0, 0.1, (n_users, self.n_factors))
            self.item_factors = np.random.normal(0, 0.1, (n_items, self.n_factors))
            
            # 训练模型
            for epoch in range(self.n_epochs):
                for i, j, rating in zip(rows, cols, values):
                    # 计算预测值
                    pred = (
                        self.global_bias + 
                        self.user_biases[i] + 
                        self.item_biases[j] + 
                        np.dot(self.user_factors[i], self.item_factors[j])
                    )
                    
                    # 计算误差
                    error = rating - pred
                    
                    # 更新参数
                    self.user_biases[i] += self.lr * (error - self.reg * self.user_biases[i])
                    self.item_biases[j] += self.lr * (error - self.reg * self.item_biases[j])
                    
                    # 更新因子
                    uf = self.user_factors[i]
                    itf = self.item_factors[j]
                    
                    self.user_factors[i] += self.lr * (error * itf - self.reg * uf)
                    self.item_factors[j] += self.lr * (error * uf - self.reg * itf)
            
            return self
        
        def predict(self, user_idx, item_idx):
            return (
                self.global_bias + 
                self.user_biases[user_idx] + 
                self.item_biases[item_idx] + 
                np.dot(self.user_factors[user_idx], self.item_factors[item_idx])
            )
        
        def recommend(self, user_idx, n=10, exclude_interacted=True):
            # 计算所有商品的预测评分
            scores = (
                self.global_bias + 
                self.user_biases[user_idx] + 
                self.item_biases + 
                np.dot(self.user_factors[user_idx], self.item_factors.T)
            )
            
            # 排除用户已经交互过的商品
            if exclude_interacted:
                interacted_items = self.interaction_matrix[user_idx].indices
                scores[interacted_items] = -np.inf
            
            # 获取top N推荐
            top_item_indices = np.argsort(scores)[::-1][:n]
            return top_item_indices

    class HybridModel:
        """混合推荐模型"""
        def __init__(self, user_cf, item_cf, svdpp, weights=(0.4, 0.3, 0.3)):
            self.user_cf = user_cf
            self.item_cf = item_cf
            self.svdpp = svdpp
            self.weights = weights
        
        def predict(self, user_id, n=10):
            # 获取各模型的推荐结果
            user_cf_recs = self.user_cf.predict(user_id, n*3)
            item_cf_recs = self.item_cf.predict(user_id, n*3)
            svdpp_recs = self.svdpp.recommend(self.svdpp.user_to_idx[user_id], n*3)
            
            # 转换为商品ID
            svdpp_recs = [self.svdpp.idx_to_item[idx] for idx in svdpp_recs]
            
            # 合并推荐结果并加权
            combined_scores = {}
            
            # 为用户协同过滤结果加权
            for i, item in enumerate(user_cf_recs):
                combined_scores[item] = combined_scores.get(item, 0) + self.weights[0] * (1.0 / (i+1))
            
            # 为物品协同过滤结果加权
            for i, item in enumerate(item_cf_recs):
                combined_scores[item] = combined_scores.get(item, 0) + self.weights[1] * (1.0 / (i+1))
            
            # 为SVD++结果加权
            for i, item in enumerate(svdpp_recs):
                combined_scores[item] = combined_scores.get(item, 0) + self.weights[2] * (1.0 / (i+1))
            
            # 排序并获取top N
            sorted_items = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:n]
            return [item for item, score in sorted_items]

    def train_models(self):
        """训练所有模型"""
        print("Training models...")
        start_time = time.time()
        
        # 创建交互矩阵
        (train_matrix, user_to_idx, item_to_idx, 
         unique_users, unique_items) = self.create_interaction_matrix(self.train_data)
        
        # 训练User-CF模型
        user_cf_start = time.time()
        self.user_cf_model = self.UserCF(k=self.config['top_k'])
        self.user_cf_model.fit(train_matrix, user_to_idx, item_to_idx)
        user_cf_time = time.time() - user_cf_start
        
        # 训练Item-CF模型
        item_cf_start = time.time()
        self.item_cf_model = self.ItemCF(k=self.config['top_k'])
        self.item_cf_model.fit(train_matrix, user_to_idx, item_to_idx)
        item_cf_time = time.time() - item_cf_start
        
        # 训练SVD++模型
        svdpp_start = time.time()
        self.svdpp_model = self.SVDpp(n_factors=self.config['svd_factors'])
        self.svdpp_model.interaction_matrix = train_matrix
        self.svdpp_model.user_to_idx = user_to_idx
        self.svdpp_model.idx_to_user = {v: k for k, v in user_to_idx.items()}
        self.svdpp_model.item_to_idx = item_to_idx
        self.svdpp_model.idx_to_item = {v: k for k, v in item_to_idx.items()}
        self.svdpp_model.fit(train_matrix)
        svdpp_time = time.time() - svdpp_start
        
        # 训练混合模型
        hybrid_start = time.time()
        self.hybrid_model = self.HybridModel(
            self.user_cf_model, 
            self.item_cf_model, 
            self.svdpp_model
        )
        hybrid_time = time.time() - hybrid_start
        
        total_time = time.time() - start_time
        
        print(f"Models trained in {total_time:.2f} seconds")
        print(f"- User-CF: {user_cf_time:.2f}s")
        print(f"- Item-CF: {item_cf_time:.2f}s")
        print(f"- SVD++: {svdpp_time:.2f}s")
        print(f"- Hybrid: {hybrid_time:.2f}s")
        
        return {
            'user_cf': user_cf_time,
            'item_cf': item_cf_time,
            'svdpp': svdpp_time,
            'hybrid': hybrid_time,
            'total': total_time
        }

    def evaluate_model(self, model, model_name):
        """评估模型性能"""
        print(f"Evaluating {model_name} model...")
        start_time = time.time()
        
        # 准备测试数据
        test_users = self.test_data['userId'].unique()
        test_data_grouped = self.test_data.groupby('userId')['itemId'].apply(set).to_dict()
        
        # 评估指标
        precisions = []
        recalls = []
        coverages = []
        all_recommended = set()
        all_items = set(self.df['itemId'].unique())
        
        # 长尾商品识别
        item_popularity = self.df['itemId'].value_counts()
        long_tail_items = set(item_popularity[item_popularity < 10].index)
        
        long_tail_ratios = []
        novel_items = set()
        
        # 对每个测试用户进行评估
        for i, user in enumerate(test_users):
            if i > 0 and i % 100 == 0:
                print(f"Processed {i}/{len(test_users)} users...")
            
            # 获取真实交互的商品
            true_items = test_data_grouped.get(user, set())
            
            # 生成推荐
            recommendations = model.predict(user, self.config['top_k'])
            
            # 计算Precision和Recall
            if recommendations:
                hits = len(set(recommendations) & true_items)
                precisions.append(hits / len(recommendations))
                recalls.append(hits / len(true_items) if true_items else 0)
            else:
                precisions.append(0)
                recalls.append(0)
            
            # 计算覆盖率
            all_recommended.update(recommendations)
            coverages.append(len(all_recommended) / len(all_items))
            
            # 计算长尾商品比例
            if recommendations:
                long_tail_count = sum(1 for item in recommendations if item in long_tail_items)
                long_tail_ratios.append(long_tail_count / len(recommendations))
                novel_items.update([item for item in recommendations if item not in true_items])
        
        # 计算平均指标
        precision_avg = np.mean(precisions)
        recall_avg = np.mean(recalls)
        coverage_avg = np.mean(coverages)
        long_tail_avg = np.mean(long_tail_ratios) if long_tail_ratios else 0
        novelty_avg = len(novel_items) / len(all_items) if all_items else 0
        
        eval_time = time.time() - start_time
        
        print(f"Evaluation completed in {eval_time:.2f} seconds")
        print(f"- Precision@{self.config['top_k']}: {precision_avg:.4f}")
        print(f"- Recall@{self.config['top_k']}: {recall_avg:.4f}")
        print(f"- Coverage@{self.config['top_k']}: {coverage_avg:.4f}")
        print(f"- Long Tail Ratio: {long_tail_avg:.4f}")
        print(f"- Novelty: {novelty_avg:.4f}")
        
        # 保存结果
        self.results[model_name] = {
            'precision': precision_avg,
            'recall': recall_avg,
            'coverage': coverage_avg,
            'long_tail': long_tail_avg,
            'novelty': novelty_avg,
            'eval_time': eval_time
        }
        
        return self.results[model_name]

    def visualize_results(self):
        """可视化评估结果"""
        print("Visualizing results...")
        
        # 准备数据
        models = list(self.results.keys())
        metrics = ['precision', 'recall', 'coverage', 'long_tail']
        metric_names = ['Precision@10', 'Recall@10', 'Coverage@10', 'Long Tail Ratio']
        
        # 创建子图
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        axes = axes.flatten()
        
        # 绘制每个指标的柱状图
        for i, metric in enumerate(metrics):
            values = [self.results[model][metric] for model in models]
            ax = axes[i]
            bars = ax.bar(models, values, color=sns.color_palette("viridis", len(models)))
            ax.set_title(metric_names[i], fontsize=14)
            ax.set_ylabel('Score', fontsize=12)
            ax.grid(axis='y', linestyle='--', alpha=0.7)
            
            # 在柱子上方添加数值
            for bar in bars:
                height = bar.get_height()
                ax.annotate(f'{height:.4f}',
                            xy=(bar.get_x() + bar.get_width() / 2, height),
                            xytext=(0, 3),  # 3 points vertical offset
                            textcoords="offset points",
                            ha='center', va='bottom')
        
        plt.tight_layout()
        plt.savefig('results/model_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 训练和评估时间对比
        time_data = {
            'Training Time': {
                'User-CF': self.results['User-CF'].get('train_time', 0),
                'Item-CF': self.results['Item-CF'].get('train_time', 0),
                'SVD++': self.results['SVD++'].get('train_time', 0),
                'Hybrid': self.results['Hybrid'].get('train_time', 0)
            },
            'Evaluation Time': {
                'User-CF': self.results['User-CF']['eval_time'],
                'Item-CF': self.results['Item-CF']['eval_time'],
                'SVD++': self.results['SVD++']['eval_time'],
                'Hybrid': self.results['Hybrid']['eval_time']
            }
        }
        
        fig, ax = plt.subplots(figsize=(12, 8))
        bar_width = 0.35
        index = np.arange(len(models))
        
        train_times = [time_data['Training Time'][model] for model in models]
        eval_times = [time_data['Evaluation Time'][model] for model in models]
        
        bar1 = ax.bar(index, train_times, bar_width, label='Training Time')
        bar2 = ax.bar(index + bar_width, eval_times, bar_width, label='Evaluation Time')
        
        ax.set_xlabel('Model', fontsize=12)
        ax.set_ylabel('Time (seconds)', fontsize=12)
        ax.set_title('Model Training and Evaluation Time', fontsize=14)
        ax.set_xticks(index + bar_width / 2)
        ax.set_xticklabels(models)
        ax.legend()
        ax.grid(axis='y', linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        plt.savefig('results/time_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 用户行为时间模式可视化
        self.df['hour'] = self.df['timestamp'].dt.hour
        hourly_counts = self.df.groupby('hour').size()
        
        plt.figure(figsize=(12, 6))
        sns.lineplot(x=hourly_counts.index, y=hourly_counts.values)
        plt.title('User Activity by Hour of Day', fontsize=14)
        plt.xlabel('Hour of Day', fontsize=12)
        plt.ylabel('Number of Interactions', fontsize=12)
        plt.xticks(range(0, 24))
        plt.grid(linestyle='--', alpha=0.7)
        plt.savefig('results/hourly_activity.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 长尾分布可视化
        item_counts = self.df['itemId'].value_counts()
        
        plt.figure(figsize=(12, 6))
        plt.plot(np.arange(len(item_counts)), item_counts.values)
        plt.title('Item Popularity Distribution (Long Tail)', fontsize=14)
        plt.xlabel('Item Rank', fontsize=12)
        plt.ylabel('Interaction Count', fontsize=12)
        plt.yscale('log')
        plt.grid(linestyle='--', alpha=0.7)
        plt.savefig('results/long_tail_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print("Visualizations saved to results/ directory")

    def run_full_pipeline(self, movielens_path, amazon_path):
        """运行完整流程"""
        # 1. 数据加载与预处理
        self.load_data(movielens_path, amazon_path)
        self.detect_anomalies()
        self.normalize_ratings()
        self.apply_time_decay()
        self.extract_text_features()
        self.filter_sparse_data()
        
        # 2. 特征工程
        self.generate_graph_embeddings()
        
        # 3. 数据划分
        self.split_data()
        
        # 4. 模型训练
        train_times = self.train_models()
        
        # 5. 模型评估
        self.evaluate_model(self.user_cf_model, "User-CF")
        self.evaluate_model(self.item_cf_model, "Item-CF")
        self.evaluate_model(self.svdpp_model, "SVD++")
        self.evaluate_model(self.hybrid_model, "Hybrid")
        
        # 添加训练时间到结果
        for model in self.results:
            self.results[model]['train_time'] = train_times[model.lower()]
        
        # 6. 结果可视化
        self.visualize_results()
        
        # 7. 保存结果
        results_df = pd.DataFrame(self.results).T
        results_df.to_csv('results/final_results.csv')
        
        print("Full pipeline completed!")
        return results_df


# 示例用法
if __name__ == "__main__":
    # 初始化推荐系统
    rec_sys = RecommendationSystem()
    
    # 设置数据集路径
    movielens_path = "data/raw/movielens_ratings.csv"
    amazon_path = "data/raw/amazon_reviews.csv"
    
    # 运行完整流程
    results = rec_sys.run_full_pipeline(movielens_path, amazon_path)
    
    # 打印最终结果
    print("\nFinal Results:")
    print(results)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag